In [330]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords # the for in with
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # Loved = [0.0]
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [331]:
news_df = pd.read_csv('train.csv.zip')

In [332]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [333]:
news_df.shape

(20800, 5)

In [334]:
news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [335]:
news_df = news_df.fillna(' ')

In [336]:
news_df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [337]:
news_df['content'] = news_df['author']+" "+news_df['title']

In [338]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [339]:
news_df['content'][20796]

'Benjamin Hoffman N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [340]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content] 
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content

In [341]:
news_df['content'] = news_df['content'].apply(stemming)

In [342]:
news_df['content']

0        darrel lucu hous dem aid we didn t even see co...
1        daniel j flynn flynn hillari clinton big woman...
2        consortiumnew com whi the truth might get you ...
3        jessica purkiss civilian kill in singl us airs...
4        howard portnoy iranian woman jail for fiction ...
                               ...                        
20795    jerom hudson rapper t i trump a poster child f...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc and rachel abram maci s i...
20798    alex ansari nato russia to hold parallel exerc...
20799                   david swanson what keep the f aliv
Name: content, Length: 20800, dtype: object

In [343]:
X = news_df['content'].values
y = news_df['label'].values

In [344]:
print(X)

['darrel lucu hous dem aid we didn t even see comey s letter until jason chaffetz tweet it'
 'daniel j flynn flynn hillari clinton big woman on campu breitbart'
 'consortiumnew com whi the truth might get you fire' ...
 'michael j de la merc and rachel abram maci s is said to receiv takeov approach by hudson s bay the new york time'
 'alex ansari nato russia to hold parallel exercis in balkan'
 'david swanson what keep the f aliv']


In [345]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [346]:
print(X)

  (np.int32(0), np.int32(272))	0.23990674506736326
  (np.int32(0), np.int32(2501))	0.3265523102775443
  (np.int32(0), np.int32(2977))	0.21925031305348405
  (np.int32(0), np.int32(3619))	0.319661529542723
  (np.int32(0), np.int32(3811))	0.24029045046636915
  (np.int32(0), np.int32(4028))	0.2635219027244823
  (np.int32(0), np.int32(4997))	0.20723456165570225
  (np.int32(0), np.int32(7041))	0.19428864959389178
  (np.int32(0), np.int32(7642))	0.1558588271295779
  (np.int32(0), np.int32(7734))	0.2201449029166544
  (np.int32(0), np.int32(8673))	0.2594685946711103
  (np.int32(0), np.int32(8953))	0.3229500947064574
  (np.int32(0), np.int32(13533))	0.22790561725543845
  (np.int32(0), np.int32(15767))	0.25300730329870835
  (np.int32(0), np.int32(16060))	0.28379135534554073
  (np.int32(0), np.int32(16591))	0.19185657544008944
  (np.int32(1), np.int32(1512))	0.29025992042009807
  (np.int32(1), np.int32(1910))	0.15325078862748648
  (np.int32(1), np.int32(2241))	0.3778771044604833
  (np.int32(1), np

In [347]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [348]:
X_train.shape

(16640, 17232)

In [349]:
X_test.shape

(4160, 17232)

In [350]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [351]:
train_y_pred = model.predict(X_train)
print("train accuracy :",accuracy_score(train_y_pred,y_train))

train accuracy : 0.9864783653846154


In [352]:
test_y_pred = model.predict(X_test)
print("train accuracy :",accuracy_score(test_y_pred,y_test))

train accuracy : 0.9752403846153846


In [353]:
# prediction system

input_data = X_test[10]
prediction =  model.predict(input_data)
if prediction[0] == 1:
    print('Fake News')
else:
    print('Real News')

Real News
